In [1]:
import os
import cv2
import tensorflow as tf
import numpy as np

In [2]:
from utilities import *
#from submodules.models.research.object_detection.utils import label_map_util

In [3]:
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
LABEL_MAP_NAME = 'mscoco_label_map.pbtxt'

CWD_PATH = os.getcwd()
MODELS_PATH = os.path.abspath(os.path.join(CWD_PATH, 'models'))
LABELS_PATH = os.path.abspath(os.path.join(CWD_PATH, 'labels'))
DATA_PATH = os.path.abspath(os.path.join(CWD_PATH, 'data'))

VIDEO_FILE = 'cows.mp4'
VIDEO_PATH = os.path.join(DATA_PATH, VIDEO_FILE)

MODEL_PATH = os.path.join(MODELS_PATH, MODEL_NAME, 'frozen_inference_graph.pb')
LABEL_MAP_PATH = os.path.join(LABELS_PATH, LABEL_MAP_NAME)

THRESHOLD = 0

In [4]:
print(MODEL_PATH)
print(LABEL_MAP_PATH)

/Users/nuhaabakry/Google Drive/codebase/sts-tensorflow/explore-tf/models/ssd_mobilenet_v1_coco_2017_11_17/frozen_inference_graph.pb
/Users/nuhaabakry/Google Drive/codebase/sts-tensorflow/explore-tf/labels/mscoco_label_map.pbtxt


In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(MODEL_PATH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [6]:
 def detect_objects(image_np, sess, detection_graph, threshold=0):
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

        # Each box represents a part of the image where a particular object was detected.
        boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

        # Each score represent level of confidence for each of the objects.
        scores = detection_graph.get_tensor_by_name('detection_scores:0')
        classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        # Actual detection.
        (boxes, scores, classes, num_detections) = sess.run(
            [boxes, scores, classes, num_detections],
            feed_dict={image_tensor: image_np_expanded})
        return (boxes, scores, classes, num_detections)
        # filter for target classes within threshold
        boxes = np.squeeze(boxes)
        classes = np.squeeze(classes)
        scores = np.squeeze(scores)
        ix_scores = scores >= threshold
        ix = ix_classes * ix_scores
        tf_data = [boxes[ix], np.extract(ix, classes), np.extract(ix, scores)]

        return tf_data

In [ ]:
# WEBCAM
#video_stream = VideoStream(src=0).start() 
# VIDEO File
video_stream = VideoStream(src=VIDEO_PATH).start()

while video_stream.grabbed:
    frame, counter = video_stream.read()
    
    # do some zoo model detection on the frame
    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:
            data = detect_objects(frame, sess, detection_graph)
    print(data)
    
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_stream.stop()
cv2.destroyAllWindows()
cv2.waitKey(1000) # to autoclose window after a few seconds

(array([[[4.3046892e-01, 6.0553789e-01, 5.7542503e-01, 6.4241588e-01],
        [3.6435923e-01, 7.8568232e-01, 5.9716040e-01, 9.7944725e-01],
        [3.6715344e-01, 4.8714370e-01, 6.1202985e-01, 5.7096690e-01],
        [4.1557619e-01, 3.3704618e-01, 5.5257523e-01, 4.4005248e-01],
        [3.6746883e-01, 7.8920704e-01, 6.1165780e-01, 9.7129196e-01],
        [4.3276897e-01, 6.2280440e-01, 5.7259893e-01, 6.6105175e-01],
        [4.1557619e-01, 3.3704618e-01, 5.5257523e-01, 4.4005248e-01],
        [4.3046892e-01, 6.0553789e-01, 5.7542503e-01, 6.4241588e-01],
        [4.1557619e-01, 3.3704618e-01, 5.5257523e-01, 4.4005248e-01],
        [4.2076689e-01, 9.3710005e-01, 5.9517282e-01, 9.9834692e-01],
        [3.6863416e-01, 7.7472281e-01, 5.8791560e-01, 8.8982964e-01],
        [4.3445185e-01, 5.3971279e-01, 5.8193988e-01, 5.7654238e-01],
        [4.7801918e-01, 5.2981287e-01, 6.0561842e-01, 5.6487542e-01],
        [4.8247617e-01, 9.4703054e-01, 6.0802609e-01, 9.9911654e-01],
        [3.6715344e

(array([[[0.3294652 , 0.5672488 , 0.58030003, 0.70976365],
        [0.39513683, 0.707777  , 0.60279036, 0.84877205],
        [0.36125717, 0.4463336 , 0.6124245 , 0.5545531 ],
        [0.36125717, 0.4463336 , 0.6124245 , 0.5545531 ],
        [0.42040655, 0.30926353, 0.5510024 , 0.40587544],
        [0.3278193 , 0.55951595, 0.583087  , 0.7078488 ],
        [0.36125717, 0.4463336 , 0.6124245 , 0.5545531 ],
        [0.33954996, 0.48176062, 0.5855532 , 0.5906054 ],
        [0.39513683, 0.707777  , 0.60279036, 0.84877205],
        [0.33954996, 0.48176062, 0.5855532 , 0.5906054 ],
        [0.50432354, 0.4484159 , 0.60331935, 0.48945174],
        [0.40221334, 0.32162753, 0.53864545, 0.42473564],
        [0.3955261 , 0.31099448, 0.58702564, 0.44162896],
        [0.35237068, 0.4777779 , 0.48410422, 0.55119187],
        [0.3276405 , 0.49663103, 0.608727  , 0.8245373 ],
        [0.39073592, 0.64724207, 0.58667874, 0.8324506 ],
        [0.4609257 , 0.431579  , 0.56036043, 0.473877  ],
        [0.40